In [4]:
library(data.table)
load('../data/exonicvsnon_counts.rda') # Probabilities of each mutation type
load('../data/gene_pd_exon.rda') # Load tables giving probability of insertion for each gene in a diploid genome for both genders
source('./src_sompop_v0_2_4.r')
xy_genes <- gene_pd_m$gene_id[gene_pd_m$chrom %in% c('X','Y')] # Getting list of genes on chroms. X and Y by symbol

In [5]:
geneList <- read.csv('../data/drivers_coread.txt',header=F)
geneList <- geneList$V1

In [6]:
gene_pd_m[gene_pd_m$gene_id %in% geneList]
geneList <- gene_pd_m[gene_pd_m$gene_id %in% geneList]$gene_sym

chrom,gene_id,counts,length,type,gene_sym,p
4,ENSG00000109670,314.9761,11095,0,FBXW7,1.640391e-04
5,ENSG00000134982,281.4618,12440,0,APC,1.491041e-04
17,ENSG00000141510,119.4996,5688,0,TP53,6.371016e-05
18,ENSG00000141646,386.5472,13123,0,SMAD4,2.008213e-04
10,ENSG00000148737,181.2369,6736,0,TCF7L2,9.473898e-05
18,ENSG00000175387,988.1301,36366,0,SMAD2,5.161720e-04
X,ENSG00000184675,107.7702,8443,0,AMER1,3.038087e-05


### Define parameters

In [7]:
###################################################3
# Gender of organism
gender <- 'male'

# Symbol of driver gene disrupted at t=0
driverGene <- ''

# List of driver genes
geneList <- geneList

# Initial number of cells
N0 <- 1e3

# Average # L1 insertions / cell cycle
mu <- 1
if (gender=='male') {
    mu <- mu*(1-pd_exvsnon_m[2])# Scale by 1-probability of null (intron) insertion, to convert to probability of exonic insertion
} else if (gender=='female') {
    mu <- mu*(1-pd_exvsnon_f[2])
}

# Selective strengths of each mutation type
spd <- 0.1 # Homozygous driver
spp <- 0.005 # Homozygous passenger
sld <- spd*0.1 # Heterozygous driver
slp <- spp*0.1 # Hetero. passenger

# Number of time steps to simulate
NT <- 5e3

###################################################3
# Time resolution: number of timesteps per generation
tau <- 1
tau <- 1/tau
# Buffer size of population data object; represents the max possible number of clones in the population
maxNClones <- N0*100
# Log path
logpath <- './test_v2.4.log'

## Test run

In [10]:
N0 <- 100
mu <- 5
sd <- 0.1
sp <- 0.005
out <- sompop(N0, mu, tau, NT, sld, slp, spd, spp, gender, driverGene, geneList, maxNClones, logpath)
Pop <- out[[1]]
N <- out[[2]]
genTime <- out[[4]]

# head(Pop)
N[N==0] <- NA
genTime[genTime==0] <- NA
plot(1:NT*tau,N,type='l',xlab='Generation',ylab='Population Size')

## Batch run

In [ ]:
nrun <- 0

N0 <- 1e3
sd <- .1
hef <- 0.1 # Fraction of homozygous effect assigned to heterozygous mutations

outPath <- '~/jackgl/colon_1113_2/'
logPath <- paste0(outPath,'run.log')

tryCatch({

    for (ii in 1:100) {
        mu <- sample(seq(0.01,6,0.05),1) # Sample random mu
        sp <- sample(exp(seq(log(0.0001),log(1),0.1)),1) # Sample random sp (from log scale)
        # Convert mu to expected number of exonic insertions per cell cycle
        if (gender=='male') {
            mu_adj <- mu*(1-pd_exvsnon_m[2])
        } else if (gender=='female') {
            mu_adj <- mu*(1-pd_exvsnon_f[2])
        }
        
        for (nn in 1:10) {
            nrun <- nrun+1
            line <- paste0('Run: ',toString(nrun),'\tN0: ',toString(N0),'\tmu: ',toString(mu),'\tsd: ',toString(sd),'\tsp: ',toString(sp))
            write(line,file=logPath,append=TRUE)
            out <- sompop(N0, mu_adj, tau, NT, sd*hef, sp*hef, sd, sp, gender, driverGene, geneList, N0*4, logPath)
            Pop <- out[[1]]
            N <- out[[2]]
            mut_genes <- out[[3]]
            gen_time <- out[[4]]
            save(N0,mu,sd,sp,Pop,N,mut_genes,gen_time,file=paste0(outPath,'run_n0',N0,'_',nrun,".rda"))
            rm(Pop,N,mut_genes,gen_time)
        }
    }

    }, warning = function(warn.msg){
        write(toString(warn.msg), logPath, append=TRUE)
    }, error = function(err.msg){
        write(toString(err.msg), logPath, append=TRUE)
    }
)